# Format a latex table of reactions according to an input spreadsheet

In [1]:
import pandas as pd
import copy

In [2]:
pd.set_option('display.max_colwidth', None)

In [3]:
main_folder = "/home/emc/OneDrive-CU/Research/UpperAtmoDH/Results/I-Ts230Tm130Te210-Wstandard-Smean_qNZibSp4_smean_v17/"

# Useful function

In [4]:
def format_spc(s):
    """
    Replace certain strings with latex strings
    """
    s = s.replace("pl", "^+")
    s = s.replace("E", "e^-")
    s = s.replace("2", "_2")
    s = s.replace("3", "_3")
    s = s.replace("up_2D", "(^2D)")
    s = s.replace("1D", "(^1D)")
    return s
    
def participant_str(r1, r2="", r3=""):
    
    # print(f"Working with {r1} and {r2}")
    if r1 != "":
        if (r2 != "") & (r2 != "none"):
            if (r3 != "") & (r3 != "none"):
                return f"{format_spc(r1)} + {format_spc(r2)} + {format_spc(r3)}"
            else:
                return f"{format_spc(r1)} + {format_spc(r2)}"
        else:
            return f"{format_spc(r1)}"
    else:
        return ""

def br_str(br):
    if (br == "") | (int(float(br)) == 1): # (int(float(br)) == 0) | 
        return "" 
    else:
        return f"{br}"

    
def f_str(f):
    if f == "":
        return "" 
    else:
        if float(f) != 0:
            return f"{f}"
        else: 
            return ""

    
def mscale_str(m2, m1, p):
    if m2 != 1:
        if p == -0.5:
            return f"$\sqrt{{ \\frac{{{m1}}}{{{m2}}} }}$"
        else:
            return f"$\left(\\frac{{{m2}}}{{{m1}}}\right)^{{{p}}}$"
    else:
        return ""
    
# Enter in the rate coefficient column
def a_str(a):
    Astring = str("{:.2e}".format(a))#Astring = str(a)
    if "e-" in Astring:
        Astring = Astring.replace("e-0", r" \times 10^{-")
        Astring = Astring.replace("e-", r" \times 10^{-")
        Astring = Astring + "}"

    return Astring

def b_str(b, T="T_i"):
    if b != 0:
        return f"\left(\\frac{{{T}}}{{300}}\right)^{{{b}}}"
    else:
        return ""

def c_str(c, T="T_i"):
    if c != 0:
        return f"e^{{{c:g}/{T}}}"
    else:
        return ""
    
def k_str(a, b, c, a0="", b0="", c0="", T="T_i"):
    if (a0 != 0) & (a0 != ""):
        return r"\makecell[l]{See text \\ k$_{\infty}="+f"{a_str(a)}{b_str(b, T=T)}{c_str(c, T=T)}$ \\\ k$_0={a_str(a0)}{b_str(b0, T=T)}{c_str(c0, T=T)}$" + r"}"
    else:
        return f"${a_str(a)}{b_str(b, T=T)}{c_str(c, T=T)}$"

def typestr(t):
    if t != "":
        if int(t) > 2:
            return str(int(t))
        else:
            return ""
    else:
        return ""
    
def refstr(r):
    r = r.replace("Mass scaling???", "Est.")
    r = r.replace("Mass scaling", "Est.")
    r = r.replace("estimated", "Est.")
    r = r.replace("estimate", "Est.")
       
    r = r.replace("Anicich2003", "A03")
    r = r.replace("Anicich93, Klippenstein2010", "A93,Kl10")
    r = r.replace("Anicich1993", "A93")
    r = r.replace("Anicich93", "A93")
    
    r = r.replace("Atkinson89", "At89")
    r = r.replace("Antipov09", "Ant09")
    
    r = r.replace("Baulch05 (H analogue rate)", "Est.")
    r = r.replace("Baulch05, Horowitz78", "B05")
    r = r.replace("Baulch05", "B05")
    r = r.replace("Baulch92", "B92")
    r = r.replace("Borodi2009", "Bo09")
    r = r.replace("Burkholder2020.", "Bu20")
    r = r.replace("Burkholder2020", "Bu20")
    r = r.replace("Burkholder2020, Chaffin2017", "Bu20")
    r = r.replace("Brune83", "Br83")
    
    r = r.replace("Campbell73", "Cam73")
    r = r.replace("Cazaux2010", "C10")
    r = r.replace("Chaffin2017", "Est.")
    
    r = r.replace("Deighan2012", "D12")
    r = r.replace("Deighan12", "D12")
    r = r.replace("Davidson78", "Da78")
    r = r.replace("Doroshenko92", "Do92")
    r = r.replace("Dutuit13", "Du13")
    
    r = r.replace("Fehsenfeld1970", "Feh70")
    r = r.replace("Fletcher76", "Fl76")
    r = r.replace("Fox1993", "F93")
    r = r.replace("estimated (Fox2015)", "F15")
    r = r.replace("Fox2015, Vejby-Christensen1998, Hellberg2003", "F15,V-C98,Hel03")
    
    r = r.replace("rate Korolov2009, BR Geppert05", "GK")
    r = r.replace("Geppert04,Herd90", "G04,H90")
    r = r.replace("Geppert05", "G05")
    r = r.replace("Geppert00", "G00")
    
    r = r.replace("Gustafsson15", "Gu15")
    r = r.replace("Heidner73", "Hei73")
    r = r.replace("Herron99", "Her99")
    r = r.replace("Hieri97", "Hi97")
    
    r = r.replace("Jensen00", "J00")
    
    r = r.replace("KIDA (Baulch2005)", "KIDA")
    
    r = r.replace("Korolov2009", "K09")
    r = r.replace("LeGarrec03", "L03")
    r = r.replace("Milligan00,Klippenstein10", "M00,Kl10")
    r = r.replace("Mitchell05", "Mi05")
    r = r.replace("Mitchell90", "Mi90")
    
    r = r.replace("Nahar97", "N97")
    r = r.replace("NIST (Shavitt 1959)", "NIST")
    r = r.replace("NIST (Mitchell & Le Roy 1973)", "NIST")
    r = r.replace("NIST apparently", "NIST")
      
    r = r.replace("Okabe78", "O78")
    
    r = r.replace("Paraskevopoulos71", "P71")
    r = r.replace("Praxmarer94", "Pr94")
    
    r = r.replace("Rosen98", "R98")
    r = r.replace("Rosen00", "R00")
    
    r = r.replace("Sander15, Burkholder2020", "Bu20")
    r = r.replace("Sander15", "S15")
    r = r.replace("Sander2011", "S11")
    r = r.replace(" Sander2011, NIST", "S11")
    r = r.replace("Scott97", "Sc97")
    r = r.replace("Scott98", "Sc98")
    r = r.replace("Schunk09", "SN09")
    r = r.replace("Smith02", "Sm02")
    r = r.replace("Spanel93", "Sp93")
    
    r = r.replace("Tully75", "Tu75")
    r = r.replace("Tsang86", "Ts86")
    r = r.replace("Tsang91", "Ts91")
    
    r = r.replace("UMIST RATE12", "UMIST")
    
    r = r.replace("Viggiano80", "V80")
    r = r.replace("Viggiano05", "V05")

    r = r.replace("Yung1988, Yung1989, NIST", "NIST")
    r = r.replace("Yung1988: 0.71xH-analogue", "Y88")
    r = r.replace("Yung1988", "Y88")
    r = r.replace("Yung 1988", "Y88")
    r = r.replace("Yung1989", "Y89")
    
    r = r.replace("Zanchet09", "Z09")

    return r
    

In [5]:
a = -100.0
b = -100.5
print(f'{a:g}')
print(f'{b:g}')
print(f'{0.0:g}')
print(f'{0:g}')

-100
-100.5
0
0


In [6]:
def a_str_new(a):

    Astring = str("{:.2e}".format(a))
    if "e-" in Astring:
        Astring = Astring.replace("e-0", r" \times 10^{-")
        Astring = Astring.replace("e-", r" \times 10^{-")
        Astring = Astring + "}"

    return Astring

a_str_new(1.851851851851e-30)

'1.85 \\times 10^{-30}'

In [7]:
def assign_reaction_numbers_old(df, division_ind=None):
    """
    division_ind: where it switches from unimolecular to bimolecular.
    """
    rxn_num = 1

    comparison_r1 = ""
    comparison_r2 = ""

    if division_ind != None:
        di=division_ind
    else:
        di=-2
        
    
    for index, row in df[0:di+1].iterrows():
        L = "a"

        # print(f"Comparing Rxn {rxn_num}: {df.loc[index, 'R1']} and {df.loc[index, 'R2']} to {comparison_r1} and {comparison_r2}")

        # For when we don't have a repeated reactant set:
        if (comparison_r1 != df.loc[index, "R1"]) | ( ("R2" in df.columns) &  (comparison_r2 != df.loc[index, "R2"]) ):
            # print("No match")
            df.loc[index, "Num"] = rxn_num # give it a number 

            # This reaction becomes the point of comaprison
            comparison_r1 = df.loc[index, "R1"]
            comparison_r2 = df.loc[index, "R2"]
            # print(f"New comparisons {comparison_r1} and {comparison_r2}")

            # prepare to move on to the next
            rxn_num += 1
            comp_ind = index
            L = "a"
        elif (comparison_r1 == df.loc[index, "R1"]) & ( ("R2" in df.columns) & (comparison_r2 == df.loc[index, "R2"])):
            # print("matching")
            branchno = index - comp_ind + 1
            # print(f"Branch number is {branchno}")
            this_num = rxn_num -1
            # print(f"Reaction number to use is {this_num}")

            # Reletter the number of the first in the set with an a
            df.loc[comp_ind, "Num"] = str(this_num) + L 

            # Give this one a b, c, or whatever is appropriate
            df.loc[index, "Num"] = str(this_num) + chr(ord(L) + branchno - 1)

            # FIx up the reactant entries 
            df.loc[index, "R1"] = ""
            L = chr(ord(L)+1)

    if division_ind != None:
        for index, row in df[division_ind+1:].iterrows():
            L = "a"

            # print(f"Comparing Rxn {rxn_num}: {df.loc[index, 'R1']} and {df.loc[index, 'R2']} to {comparison_r1} and {comparison_r2}")

            # For when we don't have a repeated reactant set:
            if (comparison_r1 != df.loc[index, "R1"]) or (("R2" in df.columns) & (comparison_r2 != df.loc[index, "R2"])):
                # print("No match")
                df.loc[index, "Num"] = rxn_num # give it a number 

                # This reaction becomes the point of comaprison
                comparison_r1 = df.loc[index, "R1"]
                comparison_r2 = df.loc[index, "R2"]
                # print(f"New comparisons {comparison_r1} and {comparison_r2}")

                # prepare to move on to the next
                rxn_num += 1
                comp_ind = index
                L = "a"
            elif (comparison_r1 == df.loc[index, "R1"]) and (("R2" in df.columns) & (comparison_r2 == df.loc[index, "R2"])):
                # print("matching")
                branchno = index - comp_ind + 1
                # print(f"Branch number is {branchno}")
                this_num = rxn_num -1
                # print(f"Reaction number to use is {this_num}")

                # Reletter the number of the first in the set with an a
                df.loc[comp_ind, "Num"] = str(this_num) + L 

                # Give this one a b, c, or whatever is appropriate
                df.loc[index, "Num"] = str(this_num) + chr(ord(L) + branchno - 1)

                # FIx up the reactant entries 
                df.loc[index, "R1"] = ""
                df.loc[index, "R2"] = ""
                L = chr(ord(L)+1)
            # print()
            
def assign_reaction_numbers(df, opt_start=1):
    """
    df: dataframe
    
    opt_start: optional number at which to start the reaction numbering, useful for stitching tables together with multiple calls
    """
    
    rxn_num = opt_start

    comp = {"R1": "", "R2": "", "R3": ""}

    di=-2
        
    # Identify which columns are present in this dataframe
    rcols = set(comp.keys()).intersection(df.columns) 
    
    for index, row in df.iterrows():
        L = "a"

        # print(f"Comparing Rxn {rxn_num}: {df.loc[index, 'R1']} and {df.loc[index, 'R2']} to {comparison_r1} and {comparison_r2}")

        reactants_this_row = [df.loc[index, r] for r in rcols]
        this_row_str = "+".join(reactants_this_row)
        to_compare = [comp[r] for r in rcols]
        join_compare = "+".join(to_compare)
        
        # print(f"comparing {to_compare} and {reactants_this_row}")

        # For when we don't have a repeated reactant set:
        if to_compare != reactants_this_row: 
            # print(f"row {index} not a match: {join_compare} != {this_row_str}")
            df.loc[index, "Num"] = rxn_num # give it a number 

            # This reaction becomes the point of comaprison: Update the dictionary with its reactant values
            for r in rcols:
                comp[r] = df.loc[index, r]

            # print(f"New comparisons {comparison_r1} and {comparison_r2}")

            # prepare to move on to the next
            rxn_num += 1
            comp_ind = index
            L = "a"
        else: # For when the reactants all match, i.e. we are looking at another branch:
            # print(f"row {index} match: {join_compare} = {this_row_str}")
            branchno = index - comp_ind + 1
            # print(f"Branch number is {branchno}")
            this_num = rxn_num - 1
            # print(f"Reaction number to use is {this_num}")

            # Reletter the number of the first in the set with an a
            df.loc[comp_ind, "Num"] = str(this_num) + L 

            # Give this one a b, c, or whatever is appropriate
            df.loc[index, "Num"] = str(this_num) + chr(ord(L) + branchno - 1)

            # Clear out the reactants because we don't need them printed again
            for r in rcols:
                df.loc[index, r] = ""
                
            L = chr(ord(L)+1)

    

# Load

In [8]:
# We read the excel file as a dataframe
df_photod = pd.read_excel(f"{main_folder}active_rxns.xlsx", sheet_name="Photodissociation", 
                    index_col=None, usecols=["R1", "P1", "P2", "BR", "Reference"])

df_photoi = pd.read_excel(f"{main_folder}active_rxns.xlsx", sheet_name="Photoionization", 
                    index_col=None, usecols=["R1", "P1", "P2", "BR", "Reference"])

df_neutrals = pd.read_excel(f"{main_folder}active_rxns.xlsx", sheet_name="Neutral reactions", 
                    index_col=None, usecols=["R1", "R2", "R3", "P1", "P2", "P3", "type", "M2", "M1", "pow", "BR", "kA", "kB", "kC", "k0A", "k0B", "k0C", "F", "Reference"])

df_ions = pd.read_excel(f"{main_folder}active_rxns.xlsx", sheet_name="Ion reactions", 
                    index_col=None, usecols=["R1", "R2", "P1", "P2", "M2", "M1", "pow", "BR", "kA", "kB", "kC", "k0A", "k0B", "k0C", "Reference"])



Look for only deuterium reactions, and add some columns to hold the reaction number and some latex to add hspace, then reset the index so the df isn't messed up:

In [84]:
df_photod_D = df_photod[ df_photod["R1"].str.contains("D") ]
df_photoi_D = df_photoi[ df_photoi["R1"].str.contains("D") ]

# Separate



## Photod + i

In [85]:
photo_df = pd.concat([df_photod_D, df_photoi_D], ignore_index=True)

# Blank out any nans for R2
photo_df[['BR']] = photo_df[['BR']].fillna("")
photo_df[['Reference']] = photo_df[['Reference']].fillna("")
photo_df['P2'] = photo_df['P2'].replace(to_replace="none", value='')

# Add reaction number column
photo_df.loc[:, "Num"] = range(1, len(photo_df)+1)
photo_df.loc[:, "Rate"] = ""
photo_df.reset_index(inplace=True)
photo_df = photo_df.drop(["index"], axis=1) # don't need this column

# Initial sort
photo_df.sort_values(["R1", "P1", "P2"], inplace=True, ignore_index=True)

# Manual sort to group by reactant 1
d = {2: 3, 3: 2, 4: 5, 5: 4, 
     7: 10, 9:7, 10:9, 11:12, 12:11}
photo_df = photo_df.rename(d).sort_index()

# Re-do the reaction number column
photo_df.loc[:, "Num"] = range(1, len(photo_df)+1)


In [86]:
# Assign reaction numbers
assign_reaction_numbers(photo_df)

print(photo_df)

# Make the strings 
photo_df = photo_df.assign(reactants = [f"$\mathrm{{{participant_str(r1)}}}$" if r1!="" else "" for r1 in photo_df["R1"]] )
photo_df = photo_df.assign(products = [f"$\mathrm{{{participant_str(p1,p2)}}}$" for (p1,p2) in zip(photo_df["P1"], photo_df["P2"])])

# Make the final df
photodf_final = photo_df.drop(["R1", "P1", "P2"], axis=1)
photodf_final = photodf_final[["Num", "reactants", "products", "Rate", "Reference"]]

# We save the dataframe into a latex table
photodf_final.style.format(escape="latex")  
photodf_final.to_latex("ReactionTables/photo_reactions.tex", index=False, label="photo_rxns", longtable=True, escape=False)

      R1     P1   P2   BR Reference Num Rate
0      D    Dpl                       1     
1    DO2     OD    O                  2     
2     HD      H    D                 3a     
3           Dpl    H  0.5            3b     
4           Hpl    D  0.5            3c     
5          HDpl                      3d     
6    HDO      D   OH                 4a     
7             H   OD                 4b     
8             H    D                 4c     
9            HD  O1D                 4d     
10          Dpl   OH  0.5            4e     
11          Hpl   OD  0.5            4f     
12        HDOpl                      4g     
13         ODpl    H  0.5            4h     
14         OHpl    D  0.5            4i     
15          Opl   HD                 4j     
16  HDO2    DO2    H                 5a     
17          HDO  O1D                 5b     
18          HO2    D                 5c     
19           OH   OD                 5d     
20    OD      O    D                 6a     
21        

/tmp/ipykernel_15646/1425723958.py:16: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  photodf_final.to_latex("ReactionTables/photo_reactions.tex", index=False, label="photo_rxns", longtable=True, escape=False)


In [87]:
photodf_final


,Num,reactants,products,Rate,Reference
0,1,$\mathrm{D}$,$\mathrm{D^+}$,,
1,2,$\mathrm{DO_2}$,$\mathrm{OD + O}$,,
2,3a,$\mathrm{HD}$,$\mathrm{H + D}$,,
3,3b,,$\mathrm{D^+ + H}$,,
4,3c,,$\mathrm{H^+ + D}$,,
5,3d,,$\mathrm{HD^+}$,,
6,4a,$\mathrm{HDO}$,$\mathrm{D + OH}$,,
7,4b,,$\mathrm{H + OD}$,,
8,4c,,$\mathrm{H + D}$,,
9,4d,,$\mathrm{HD + O(^1D)}$,,


## Neutrals

In [30]:
all_species = []

from openpyxl import load_workbook

wb = load_workbook(filename = main_folder + "PARAMETERS.xlsx")
ws = wb["SpeciesLists"]
for col in ws.iter_cols(min_row=2, max_col=1, max_row=100):
    for cell in col:
        if cell.value != None:
            all_species.append(cell.value)

In [31]:
print(all_species)

['Ar', 'ArDpl', 'ArHpl', 'Arpl', 'C', 'CHpl', 'CO', 'CO2', 'CO2pl', 'COpl', 'Cpl', 'D', 'DCO', 'DCO2pl', 'DCOpl', 'DO2', 'DOCO', 'DOCpl', 'Dpl', 'H', 'H2', 'H2DOpl', 'H2Dpl', 'H2O', 'H2O2', 'H2Opl', 'H2pl', 'H3Opl', 'H3pl', 'HCO', 'HCO2pl', 'HCOpl', 'HD', 'HDO', 'HDO2', 'HDOpl', 'HDpl', 'HNOpl', 'HO2', 'HO2pl', 'HOCO', 'HOCpl', 'Hpl', 'N', 'N2', 'N2Dpl', 'N2Hpl', 'N2pl', 'NHpl', 'NO', 'NOpl', 'Npl', 'Nup2D', 'O', 'O1D', 'O2', 'O2pl', 'O3', 'OD', 'ODpl', 'OH', 'OHpl', 'Opl']


In [32]:
D_species = [a  for a in all_species if (("D" in a) & (a not in ["O1D", "Nup2D"]))]
non_D_species = list(sorted(set(all_species).difference(set(D_species))))

print(D_species)
print()
print(non_D_species)

['ArDpl', 'D', 'DCO', 'DCO2pl', 'DCOpl', 'DO2', 'DOCO', 'DOCpl', 'Dpl', 'H2DOpl', 'H2Dpl', 'HD', 'HDO', 'HDO2', 'HDOpl', 'HDpl', 'N2Dpl', 'OD', 'ODpl']

['Ar', 'ArHpl', 'Arpl', 'C', 'CHpl', 'CO', 'CO2', 'CO2pl', 'COpl', 'Cpl', 'H', 'H2', 'H2O', 'H2O2', 'H2Opl', 'H2pl', 'H3Opl', 'H3pl', 'HCO', 'HCO2pl', 'HCOpl', 'HNOpl', 'HO2', 'HO2pl', 'HOCO', 'HOCpl', 'Hpl', 'N', 'N2', 'N2Hpl', 'N2pl', 'NHpl', 'NO', 'NOpl', 'Npl', 'Nup2D', 'O', 'O1D', 'O2', 'O2pl', 'O3', 'OH', 'OHpl', 'Opl']


In [91]:
df_neutrals_D = df_neutrals[ df_neutrals["R1"].isin(D_species) | df_neutrals["R2"].isin(D_species) | df_neutrals["R3"].isin(D_species) ]


In [92]:
n_df = copy.deepcopy(df_neutrals_D)

# Blank out any nans
n_df.loc[:, 'R2'] = n_df.loc[:, 'R2'].fillna("")
n_df.loc[:, 'R3'] = n_df.loc[:, 'R3'].fillna("")
n_df.loc[:, 'P2'] = n_df.loc[:, 'P2'].fillna("")
n_df.loc[:, 'P3'] = n_df.loc[:, 'P3'].fillna("")
n_df.loc[:, 'type'] = n_df.loc[:, 'type'].fillna("")
n_df.loc[:, 'M2'] = n_df.loc[:, 'M2'].fillna("")
n_df.loc[:, 'M1'] = n_df.loc[:, 'M1'].fillna("")
n_df.loc[:, 'pow'] = n_df.loc[:, 'pow'].fillna("")
n_df.loc[:, 'BR'] = n_df.loc[:, 'BR'].fillna("")
n_df.loc[:, 'kA'] = n_df.loc[:, 'kA'].fillna("")
n_df.loc[:, 'kB'] = n_df.loc[:, 'kB'].fillna("")
n_df.loc[:, 'kC'] = n_df.loc[:, 'kC'].fillna("")
n_df.loc[:, 'k0A'] = n_df.loc[:, 'k0A'].fillna("")
n_df.loc[:, 'k0B'] = n_df.loc[:, 'k0B'].fillna("")
n_df.loc[:, 'k0C'] = n_df.loc[:, 'k0C'].fillna("")
n_df.loc[:, 'F'] = n_df.loc[:, 'F'].fillna("")


n_df.sort_values(["R1", "R2", "R3", "P1", "P2"], inplace=True, ignore_index=True)

In [93]:
# Add reaction number column
n_df.loc[:, "Num"] = range(1, len(n_df)+1)
n_df.reset_index(inplace=True)
n_df = n_df.drop(["index"], axis=1)

# Assign reaction numbers
assign_reaction_numbers(n_df, opt_start=7)

In [94]:
n_df

,R1,R2,R3,P1,P2,P3,type,M2,M1,pow,BR,kA,kB,kC,k0A,k0B,k0C,F,Reference,Num
0,CO,D,none,DCO,none,none,4,1,1,0.0,1.0,1.000000e+00,0.20,0.0,2.000000e-35,0.2,0,0.0,Baulch05 (H analogue rate),7
1,CO,OD,none,CO2,D,none,6,18,17,-0.5,1.0,1.628470e-06,6.10,0.0,4.895742e-15,0.6,0,0.0,Mass scaling,8a
2,,,,DOCO,none,none,5,18,17,-0.5,1.0,6.624225e-16,1.30,0.0,1.733093e-29,-1.4,0,0.0,Mass scaling,8b
3,D,H2,none,HD,H,none,2,1,1,0.0,1.0,2.725102e-17,2.00,-2700.0,0.000000e+00,0.0,0,0.0,NIST (Mitchell & Le Roy 1973),9
4,D,H2O2,none,H2O,OD,none,2,1,1,0.0,0.5,1.160000e-11,0.00,-2110.0,0.000000e+00,0.0,0,0.0,Cazaux2010,10a
5,,,,HDO,OH,none,2,1,1,0.0,0.5,1.160000e-11,0.00,-2110.0,0.000000e+00,0.0,0,0.0,Cazaux2010,10b
6,D,HO2,none,DO2,H,none,2,1,1,0.0,1.0,1.000000e-10,0.00,0.0,0.000000e+00,0.0,0,0.0,Yung1988,11a
7,,,,HD,O2,none,2,1,1,0.0,1.0,2.449500e-12,0.00,0.0,0.000000e+00,0.0,0,0.0,Yung1988: 0.71xH-analogue,11b
8,,,,HDO,O1D,none,2,1,1,0.0,1.0,1.136000e-12,0.00,0.0,0.000000e+00,0.0,0,0.0,Yung1988: 0.71xH-analogue,11c
9,,,,OH,OD,none,2,1,1,0.0,1.0,5.112000e-11,0.00,0.0,0.000000e+00,0.0,0,0.0,Yung1988: 0.71xH-analogue,11d


In [95]:
# Make the strings 
n_df = n_df.assign(reactants = [f"$\mathrm{{{participant_str(r1,r2,r3)}}}$" if r1!="" else "" for (r1,r2,r3) in zip(n_df["R1"], n_df["R2"], n_df["R3"])])
n_df = n_df.assign(products = [f"$\mathrm{{{participant_str(p1,p2,p3)}}}$" if p1!="" else ""  for (p1,p2,p3) in zip(n_df["P1"], n_df["P2"], n_df["P3"])])

# Other column strings
n_df = n_df.assign(ratecoeff = [k_str(A,B,C,a0=A0,b0=B0,c0=C0,T="T_n") for (A,B,C,A0,B0,C0) in zip(n_df["kA"], n_df["kB"], n_df["kC"], n_df["k0A"], n_df["k0B"], n_df["k0C"])])
n_df = n_df.assign(type = [f"{typestr(t)}" for t in n_df["type"]])
n_df = n_df.assign(BR = [f"{br_str(BR)}" for BR in n_df["BR"]])
n_df = n_df.assign(MS = [f"{mscale_str(M2, M1, p)}" for (M2, M1, p) in zip(n_df["M2"], n_df["M1"], n_df["pow"])])
n_df = n_df.assign(Ref = [f"{refstr(r)}" for r in n_df["Reference"]])
# n_df = n_df.assign(F = [f"{f_str(f)}" for f in n_df["F"]]) # No D reactions have troe parameter.

# Make the final df
n_df.rename(columns = {'ratecoeff':'Rate coefficient'}, inplace = True)
ndf_final = n_df.drop(["R1", "R2", "R3", "P1", "P2", "P3", "M2", "M1", "pow",  "kA", "kB", "kC", "k0A", "k0B", "k0C", "Reference"], axis=1)
ndf_final = ndf_final[["Num", "reactants", "products", "BR", "MS", "Rate coefficient", "Ref"]]

# We save the dataframe into a latex table
ndf_final.style.format(escape="latex")  
ndf_final.to_latex("ReactionTables/neutral_reactions.tex", index=False, label="neutral_rxns", longtable=True, escape=False)

/tmp/ipykernel_15646/2565441828.py:20: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  ndf_final.to_latex("ReactionTables/neutral_reactions.tex", index=False, label="neutral_rxns", longtable=True, escape=False)


In [96]:
ndf_final[0:30]

,Num,reactants,products,BR,MS,Rate coefficient,Ref
0,7,$\mathrm{CO + D}$,$\mathrm{DCO}$,,,\makecell[l]{See text \\ k$_{\infty}=1.00e+00\left(\frac{T_n}{300}\right)^{0.2}$ \\ k$_0=2.00 \times 10^{-35}\left(\frac{T_n}{300}\right)^{0.2}$},Est.
1,8a,$\mathrm{CO + OD}$,$\mathrm{CO_2 + D}$,,$\sqrt{ \frac{17}{18} }$,\makecell[l]{See text \\ k$_{\infty}=1.63 \times 10^{-6}\left(\frac{T_n}{300}\right)^{6.1}$ \\ k$_0=4.90 \times 10^{-15}\left(\frac{T_n}{300}\right)^{0.6}$},Est.
2,8b,,$\mathrm{DOCO}$,,$\sqrt{ \frac{17}{18} }$,\makecell[l]{See text \\ k$_{\infty}=6.62 \times 10^{-16}\left(\frac{T_n}{300}\right)^{1.3}$ \\ k$_0=1.73 \times 10^{-29}\left(\frac{T_n}{300}\right)^{-1.4}$},Est.
3,9,$\mathrm{D + H_2}$,$\mathrm{HD + H}$,,,$2.73 \times 10^{-17}\left(\frac{T_n}{300}\right)^{2.0}e^{-2700/T_n}$,NIST
4,10a,$\mathrm{D + H_2O_2}$,$\mathrm{H_2O + OD}$,0.5,,$1.16 \times 10^{-11}e^{-2110/T_n}$,C10
5,10b,,$\mathrm{HDO + OH}$,0.5,,$1.16 \times 10^{-11}e^{-2110/T_n}$,C10
6,11a,$\mathrm{D + HO_2}$,$\mathrm{DO_2 + H}$,,,$1.00 \times 10^{-10}$,Y88
7,11b,,$\mathrm{HD + O_2}$,,,$2.45 \times 10^{-12}$,Y88
8,11c,,$\mathrm{HDO + O(^1D)}$,,,$1.14 \times 10^{-12}$,Y88
9,11d,,$\mathrm{OH + OD}$,,,$5.11 \times 10^{-11}$,Y88


## Ions

In [106]:
df_ions_D = df_ions[ df_ions["R1"].isin(D_species) | df_ions["R2"].isin(D_species) ]

In [107]:
i_df = copy.deepcopy(df_ions_D)

# Blank out any nans for R2
i_df.loc[:, 'R2'] = i_df.loc[:, 'R2'].fillna("")
i_df.loc[:, 'P2'] = i_df.loc[:, 'P2'].fillna("")
i_df.loc[:, 'M2'] = i_df.loc[:, 'M2'].fillna("")
i_df.loc[:, 'M1'] = i_df.loc[:, 'M1'].fillna("")
i_df.loc[:, 'pow'] = i_df.loc[:, 'pow'].fillna("")
i_df.loc[:, 'BR'] = i_df.loc[:, 'BR'].fillna("")
i_df.loc[:, 'kA'] = i_df.loc[:, 'kA'].fillna("")
i_df.loc[:, 'kB'] = i_df.loc[:, 'kB'].fillna("")
i_df.loc[:, 'kC'] = i_df.loc[:, 'kC'].fillna("")

i_df.sort_values(["R1", "R2", "P1", "P2"], inplace=True, ignore_index=True)

i_df

,R1,R2,P1,P2,M2,M1,pow,BR,kA,kB,kC,k0A,k0B,k0C,Reference
0,ArDpl,CO,DCOpl,Ar,1,1,0.0,1.00,1.250000e-09,0.0,0.0,0,0,0,Anicich2003
1,ArDpl,CO2,DCO2pl,Ar,1,1,0.0,1.00,1.100000e-09,0.0,0.0,0,0,0,Anicich2003
2,ArDpl,H2,ArHpl,HD,1,1,0.0,1.00,4.500000e-10,0.0,0.0,0,0,0,Anicich2003
3,ArDpl,H2,H2Dpl,Ar,1,1,0.0,1.00,8.800000e-10,0.0,0.0,0,0,0,Anicich2003
4,ArDpl,N2,N2Dpl,Ar,1,1,0.0,1.00,6.000000e-10,0.0,0.0,0,0,0,Anicich2003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,OHpl,HD,HDOpl,H,3,2,-0.5,1.00,9.700000e-10,0.0,0.0,0,0,0,Mass scaling
121,Opl,D,Dpl,O,2,1,-0.5,1.00,6.400000e-10,0.0,0.0,0,0,0,Mass scaling
122,Opl,HD,ODpl,H,1,1,0.0,0.46,1.250000e-09,0.0,0.0,0,0,0,Anicich2003
123,Opl,HD,OHpl,D,1,1,0.0,0.54,1.250000e-09,0.0,0.0,0,0,0,Anicich2003


In [108]:
# Add reaction number column
i_df.loc[:, "Num"] = range(1, len(i_df)+1)
i_df.reset_index(inplace=True)
i_df = i_df.drop(["index"], axis=1)

# Assign reaction numbers
assign_reaction_numbers(i_df, opt_start=50)

In [109]:
i_df

,R1,R2,P1,P2,M2,M1,pow,BR,kA,kB,kC,k0A,k0B,k0C,Reference,Num
0,ArDpl,CO,DCOpl,Ar,1,1,0.0,1.00,1.250000e-09,0.0,0.0,0,0,0,Anicich2003,50
1,ArDpl,CO2,DCO2pl,Ar,1,1,0.0,1.00,1.100000e-09,0.0,0.0,0,0,0,Anicich2003,51
2,ArDpl,H2,ArHpl,HD,1,1,0.0,1.00,4.500000e-10,0.0,0.0,0,0,0,Anicich2003,52a
3,,,H2Dpl,Ar,1,1,0.0,1.00,8.800000e-10,0.0,0.0,0,0,0,Anicich2003,52b
4,ArDpl,N2,N2Dpl,Ar,1,1,0.0,1.00,6.000000e-10,0.0,0.0,0,0,0,Anicich2003,53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,,,HDOpl,H,3,2,-0.5,1.00,9.700000e-10,0.0,0.0,0,0,0,Mass scaling,122b
121,Opl,D,Dpl,O,2,1,-0.5,1.00,6.400000e-10,0.0,0.0,0,0,0,Mass scaling,123
122,Opl,HD,ODpl,H,1,1,0.0,0.46,1.250000e-09,0.0,0.0,0,0,0,Anicich2003,124a
123,,,OHpl,D,1,1,0.0,0.54,1.250000e-09,0.0,0.0,0,0,0,Anicich2003,124b


In [110]:
# Make the strings 
i_df = i_df.assign(reactants = [f"$\mathrm{{{participant_str(r1,r2)}}}$" for (r1,r2) in zip(i_df["R1"], i_df["R2"])])
i_df = i_df.assign(products = [f"$\mathrm{{{participant_str(p1,p2)}}}$" for (p1,p2) in zip(i_df["P1"], i_df["P2"])])

# Other column strings
i_df = i_df.assign(ratecoeff = [k_str(A,B,C) for (A,B,C) in zip(i_df["kA"], i_df["kB"], i_df["kC"])])
i_df = i_df.assign(BR = [f"{br_str(BR)}" for BR in i_df["BR"]])
i_df = i_df.assign(MS = [f"{mscale_str(M2, M1, p)}" for (M2, M1, p) in zip(i_df["M2"], i_df["M1"], i_df["pow"])])
i_df = i_df.assign(Ref = [f"{refstr(r)}" for r in i_df["Reference"]])

# Make the final df
i_df.rename(columns = {'ratecoeff':'Rate coefficient'}, inplace = True)
idf_final = i_df.drop(["R1", "R2", "P1", "P2", "M2", "M1", "pow",  "kA", "kB", "kC", "Reference"], axis=1)
idf_final = idf_final[["Num", "reactants", "products",  "BR", "MS", "Rate coefficient", "Ref"]]

# We save the dataframe into a latex table
idf_final.style.format(escape="latex")  
idf_final.to_latex("ReactionTables/ion_reactions.tex", index=False, label="ion_rxns", longtable=True, escape=False)

/tmp/ipykernel_15646/3190654.py:18: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  idf_final.to_latex("ReactionTables/ion_reactions.tex", index=False, label="ion_rxns", longtable=True, escape=False)


In [111]:
idf_final[24:50]

,Num,reactants,products,BR,MS,Rate coefficient,Ref
24,62,$\mathrm{C^+ + HD}$,$\mathrm{CH^+ + D}$,0.17,,$1.20 \times 10^{-16}$,A03
25,63a,$\mathrm{C^+ + HDO}$,$\mathrm{DCO^+ + H}$,0.5,$\sqrt{ \frac{18}{19} }$,$7.80 \times 10^{-9}\left(\frac{T_i}{300}\right)^{-0.5}$,Est.
26,63b,$\mathrm{}$,$\mathrm{DOC^+ + H}$,0.5,$\sqrt{ \frac{18}{19} }$,$1.08 \times 10^{-9}$,Est.
27,63c,$\mathrm{}$,$\mathrm{HCO^+ + D}$,0.5,$\sqrt{ \frac{18}{19} }$,$7.80 \times 10^{-9}\left(\frac{T_i}{300}\right)^{-0.5}$,Est.
28,63d,$\mathrm{}$,$\mathrm{HDO^+ + C}$,,,$2.34 \times 10^{-10}$,Est.???
29,63e,$\mathrm{}$,$\mathrm{HOC^+ + D}$,0.5,$\sqrt{ \frac{18}{19} }$,$1.08 \times 10^{-9}$,Est.
30,64,$\mathrm{DCO_2^+ + CO}$,$\mathrm{DCO^+ + CO_2}$,,$\sqrt{ \frac{45}{46} }$,$7.80 \times 10^{-10}$,Est.
31,65a,$\mathrm{DCO_2^+ + e^-}$,$\mathrm{CO + O}$,0.68,,$4.62 \times 10^{-5}\left(\frac{T_i}{300}\right)^{-0.64}$,G05
32,65b,$\mathrm{}$,$\mathrm{CO + OD}$,0.27,,$4.62 \times 10^{-5}\left(\frac{T_i}{300}\right)^{-0.64}$,G05
33,65c,$\mathrm{}$,$\mathrm{CO_2 + D}$,0.05,,$4.62 \times 10^{-5}\left(\frac{T_i}{300}\right)^{-0.64}$,G05


# Non-D reactions

In [23]:
# We read the excel file as a dataframe
df_photod = pd.read_excel(f"{main_folder}active_rxns.xlsx", sheet_name="Photodissociation", 
                    index_col=None, usecols=["R1", "P1", "P2", "P3", "BR", "Reference"])

df_photoi = pd.read_excel(f"{main_folder}active_rxns.xlsx", sheet_name="Photoionization", 
                    index_col=None, usecols=["R1", "P1", "P2", "BR", "Reference"])

df_neutrals = pd.read_excel(f"{main_folder}active_rxns.xlsx", sheet_name="Neutral reactions", 
                    index_col=None, usecols=["R1", "R2", "R3", "P1", "P2", "P3", "type", "M2", "M1", "pow", "BR", "kA", "kB", "kC", "k0A", "k0B", "k0C", "F", "Reference"])

df_ions = pd.read_excel(f"{main_folder}active_rxns.xlsx", sheet_name="Ion reactions", 
                    index_col=None, usecols=["R1", "R2", "P1", "P2", "M2", "M1", "pow", "BR", "kA", "kB", "kC", "k0A", "k0B", "k0C", "Reference"])



## Photo

In [42]:
df_photoi

,R1,P1,P2,BR,Reference
0,CO,C,Opl,NaN,NaN
1,CO,COpl,none,NaN,NaN
2,CO,O,Cpl,NaN,NaN
3,CO2,CO2pl,none,NaN,NaN
4,CO2,COpl,O,NaN,NaN
5,CO2,Cpl,O2,NaN,NaN
6,CO2,Opl,CO,NaN,NaN
7,D,Dpl,none,NaN,NaN
8,H,Hpl,none,NaN,NaN
9,H2,H2pl,none,NaN,NaN


In [24]:
df_photod_noD = df_photod[ ~df_photod["R1"].str.contains("D") ]
df_photoi_noD = df_photoi[ ~df_photoi["R1"].str.contains("D") ]

photo_df = pd.concat([df_photod_noD, df_photoi_noD], ignore_index=True)

# Blank out any nans and nones
photo_df[['BR']] = photo_df[['BR']].fillna("")
photo_df[['Reference']] = photo_df[['Reference']].fillna("")
photo_df['P2'] = photo_df['P2'].replace(to_replace="none", value='')
photo_df['P3'] = photo_df['P3'].replace(to_replace="none", value='')
photo_df[['P3']] = photo_df[['P3']].fillna("")

# Add reaction number column
photo_df.loc[:, "Num"] = range(1, len(photo_df)+1)
photo_df.loc[:, "Rate"] = ""
photo_df.reset_index(inplace=True)
photo_df = photo_df.drop(["index"], axis=1) # don't need this column

# Initial sort
photo_df.sort_values(["R1", "P1", "P2", "P3"], inplace=True, ignore_index=True)
photo_df

,R1,P1,P2,P3,BR,Reference,Num,Rate
0,CO,C,O,,,,1,
1,CO,C,Opl,,,,22,
2,CO,COpl,,,,,23,
3,CO,O,Cpl,,,,24,
4,CO2,C,O,O,,,2,
5,CO2,C,O2,,,,3,
6,CO2,CO,O,,,,4,
7,CO2,CO,O1D,,,,5,
8,CO2,CO2pl,,,,,25,
9,CO2,COpl,O,,,,26,


In [25]:
# Re-do the reaction number column
photo_df.loc[:, "Num"] = range(1, len(photo_df)+1)

# Assign reaction numbers
assign_reaction_numbers(photo_df)

photo_df

,R1,P1,P2,P3,BR,Reference,Num,Rate
0,CO,C,O,,,,1a,
1,,C,Opl,,,,1b,
2,,COpl,,,,,1c,
3,,O,Cpl,,,,1d,
4,CO2,C,O,O,,,2a,
5,,C,O2,,,,2b,
6,,CO,O,,,,2c,
7,,CO,O1D,,,,2d,
8,,CO2pl,,,,,2e,
9,,COpl,O,,,,2f,


In [26]:
# Make the strings 
photo_df = photo_df.assign(reactants = [f"$\mathrm{{{participant_str(r1)}}}$" if r1!="" else "" for r1 in photo_df["R1"]] )
photo_df = photo_df.assign(products = [f"$\mathrm{{{participant_str(p1,p2,p3)}}}$" for (p1,p2,p3) in zip(photo_df["P1"], photo_df["P2"], photo_df["P3"])])

# Make the final df
photodf_final = photo_df.drop(["R1", "P1", "P2", "P3"], axis=1)
photodf_final = photodf_final[["Num", "reactants", "products", "Rate", "Reference"]]

# We save the dataframe into a latex table
photodf_final.style.format(escape="latex")  
photodf_final.to_latex("ReactionTables/photo_reactions_nonD.tex", index=False, label="photo_rxns", longtable=True, escape=False)


/tmp/ipykernel_31114/3090603328.py:11: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  photodf_final.to_latex("ReactionTables/photo_reactions_nonD.tex", index=False, label="photo_rxns", longtable=True, escape=False)


In [27]:
photodf_final

,Num,reactants,products,Rate,Reference
0,1a,$\mathrm{CO}$,$\mathrm{C + O}$,,
1,1b,,$\mathrm{C + O^+}$,,
2,1c,,$\mathrm{CO^+}$,,
3,1d,,$\mathrm{O + C^+}$,,
4,2a,$\mathrm{CO_2}$,$\mathrm{C + O + O}$,,
5,2b,,$\mathrm{C + O_2}$,,
6,2c,,$\mathrm{CO + O}$,,
7,2d,,$\mathrm{CO + O(^1D)}$,,
8,2e,,$\mathrm{CO_2^+}$,,
9,2f,,$\mathrm{CO^+ + O}$,,


## Neutrals

In [33]:
df_neutrals_noD = df_neutrals[ ~df_neutrals["R1"].isin(D_species) & ~df_neutrals["R2"].isin(D_species) & ~df_neutrals["R3"].isin(D_species) ]

In [36]:
# Next number of reaction to start at
I = 14

n_df = copy.deepcopy(df_neutrals_noD)

# Blank out any nans
n_df.loc[:, 'R2'] = n_df.loc[:, 'R2'].fillna("")
n_df.loc[:, 'R3'] = n_df.loc[:, 'R3'].fillna("")
n_df.loc[:, 'P2'] = n_df.loc[:, 'P2'].fillna("")
n_df.loc[:, 'P3'] = n_df.loc[:, 'P3'].fillna("")
n_df.loc[:, 'type'] = n_df.loc[:, 'type'].fillna("")
n_df.loc[:, 'M2'] = n_df.loc[:, 'M2'].fillna("")
n_df.loc[:, 'M1'] = n_df.loc[:, 'M1'].fillna("")
n_df.loc[:, 'pow'] = n_df.loc[:, 'pow'].fillna("")
n_df.loc[:, 'BR'] = n_df.loc[:, 'BR'].fillna("")
n_df.loc[:, 'kA'] = n_df.loc[:, 'kA'].fillna("")
n_df.loc[:, 'kB'] = n_df.loc[:, 'kB'].fillna("")
n_df.loc[:, 'kC'] = n_df.loc[:, 'kC'].fillna("")
n_df.loc[:, 'k0A'] = n_df.loc[:, 'k0A'].fillna("")
n_df.loc[:, 'k0B'] = n_df.loc[:, 'k0B'].fillna("")
n_df.loc[:, 'k0C'] = n_df.loc[:, 'k0C'].fillna("")
n_df.loc[:, 'F'] = n_df.loc[:, 'F'].fillna("")


n_df.sort_values(["R1", "R2", "R3", "P1", "P2"], inplace=True, ignore_index=True)

# Add reaction number column
n_df.loc[:, "Num"] = range(1, len(n_df)+1)
n_df.reset_index(inplace=True)
n_df = n_df.drop(["index"], axis=1)

# Assign reaction numbers
assign_reaction_numbers(n_df, opt_start=I)

# Make the strings 
n_df = n_df.assign(reactants = [f"$\mathrm{{{participant_str(r1,r2,r3)}}}$" if r1!="" else "" for (r1,r2,r3) in zip(n_df["R1"], n_df["R2"], n_df["R3"])])
n_df = n_df.assign(products = [f"$\mathrm{{{participant_str(p1,p2,p3)}}}$" if p1!="" else ""  for (p1,p2,p3) in zip(n_df["P1"], n_df["P2"], n_df["P3"])])

# Other column strings
n_df = n_df.assign(ratecoeff = [k_str(A,B,C,a0=A0,b0=B0,c0=C0,T="T_n") for (A,B,C,A0,B0,C0) in zip(n_df["kA"], n_df["kB"], n_df["kC"], n_df["k0A"], n_df["k0B"], n_df["k0C"])])
n_df = n_df.assign(type = [f"{typestr(t)}" for t in n_df["type"]])
n_df = n_df.assign(BR = [f"{br_str(BR)}" for BR in n_df["BR"]])
n_df = n_df.assign(MS = [f"{mscale_str(M2, M1, p)}" for (M2, M1, p) in zip(n_df["M2"], n_df["M1"], n_df["pow"])])
n_df = n_df.assign(Ref = [f"{refstr(r)}" for r in n_df["Reference"]])
n_df = n_df.assign(F = [f"{f_str(f)}" for f in n_df["F"]]) # No D reactions have troe parameter.

# Make the final df
n_df.rename(columns = {'ratecoeff':'Rate coefficient'}, inplace = True)
ndf_final = n_df.drop(["R1", "R2", "R3", "P1", "P2", "P3", "M2", "M1", "pow",  "kA", "kB", "kC", "k0A", "k0B", "k0C", "Reference"], axis=1)
ndf_final = ndf_final[["Num", "reactants", "products", "BR", "MS", "Rate coefficient", "F", "Ref"]]

# We save the dataframe into a latex table
ndf_final.style.format(escape="latex")  
ndf_final.to_latex("ReactionTables/neutral_reactions_noD.tex", index=False, label="neutral_rxns", longtable=True, escape=False)

/tmp/ipykernel_31114/4019794548.py:54: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  ndf_final.to_latex("ReactionTables/neutral_reactions_noD.tex", index=False, label="neutral_rxns", longtable=True, escape=False)


In [37]:
ndf_final

,Num,reactants,products,BR,MS,Rate coefficient,F,Ref
0,14,$\mathrm{CO + H}$,$\mathrm{HCO}$,,,\makecell[l]{See text \\ k$_{\infty}=1.00e+00\left(\frac{T_n}{300}\right)^{0.2}$ \\ k$_0=2.00 \times 10^{-35}\left(\frac{T_n}{300}\right)^{0.2}$},,B05
1,15,$\mathrm{CO + O}$,$\mathrm{CO_2}$,,,\makecell[l]{See text \\ k$_{\infty}=1.00e+00e^{-1509/T_n}$ \\ k$_0=1.70 \times 10^{-33}e^{-1509/T_n}$},0.4,Ts86
2,16a,$\mathrm{CO + OH}$,$\mathrm{CO_2 + H}$,,,\makecell[l]{See text \\ k$_{\infty}=1.63 \times 10^{-6}\left(\frac{T_n}{300}\right)^{6.1}$ \\ k$_0=4.90 \times 10^{-15}\left(\frac{T_n}{300}\right)^{0.6}$},,S11
3,16b,,$\mathrm{HOCO}$,,,\makecell[l]{See text \\ k$_{\infty}=6.62 \times 10^{-16}\left(\frac{T_n}{300}\right)^{1.3}$ \\ k$_0=1.73 \times 10^{-29}\left(\frac{T_n}{300}\right)^{-1.4}$},,S15
4,17,$\mathrm{CO_2 + H}$,$\mathrm{CO + OH}$,,,$3.38 \times 10^{-10}e^{-13163/T_n}$,,UMIST
...,...,...,...,...,...,...,...,...
82,86,$\mathrm{OH + H_2O_2}$,$\mathrm{H_2O + HO_2}$,,,$2.90 \times 10^{-12}e^{-160/T_n}$,,"NIST, KIDA"
83,87,$\mathrm{OH + N}$,$\mathrm{NO + H}$,,,$1.80 \times 10^{-10}\left(\frac{T_n}{300}\right)^{-0.2}$,,B05
84,88,$\mathrm{OH + O}$,$\mathrm{O_2 + H}$,,,$1.80 \times 10^{-11}e^{180/T_n}$,,S15
85,89a,$\mathrm{OH + OH}$,$\mathrm{H_2O + O}$,,,$1.80 \times 10^{-12}$,,S15


## Ions

In [38]:
df_ions_noD = df_ions[ ~df_ions["R1"].isin(D_species) & ~df_ions["R2"].isin(D_species) ]

In [39]:
# Number to start at 
I = 90

i_df = copy.deepcopy(df_ions_noD)

# Blank out any nans for R2
i_df.loc[:, 'R2'] = i_df.loc[:, 'R2'].fillna("")
i_df.loc[:, 'P2'] = i_df.loc[:, 'P2'].fillna("")
i_df.loc[:, 'M2'] = i_df.loc[:, 'M2'].fillna("")
i_df.loc[:, 'M1'] = i_df.loc[:, 'M1'].fillna("")
i_df.loc[:, 'pow'] = i_df.loc[:, 'pow'].fillna("")
i_df.loc[:, 'BR'] = i_df.loc[:, 'BR'].fillna("")
i_df.loc[:, 'kA'] = i_df.loc[:, 'kA'].fillna("")
i_df.loc[:, 'kB'] = i_df.loc[:, 'kB'].fillna("")
i_df.loc[:, 'kC'] = i_df.loc[:, 'kC'].fillna("")

i_df.sort_values(["R1", "R2", "P1", "P2"], inplace=True, ignore_index=True)

# Add reaction number column
i_df.loc[:, "Num"] = range(1, len(i_df)+1)
i_df.reset_index(inplace=True)
i_df = i_df.drop(["index"], axis=1)

# Assign reaction numbers
assign_reaction_numbers(i_df, opt_start=I)

# Make the strings 
i_df = i_df.assign(reactants = [f"$\mathrm{{{participant_str(r1,r2)}}}$" for (r1,r2) in zip(i_df["R1"], i_df["R2"])])
i_df = i_df.assign(products = [f"$\mathrm{{{participant_str(p1,p2)}}}$" for (p1,p2) in zip(i_df["P1"], i_df["P2"])])

# Other column strings
i_df = i_df.assign(ratecoeff = [k_str(A,B,C) for (A,B,C) in zip(i_df["kA"], i_df["kB"], i_df["kC"])])
i_df = i_df.assign(BR = [f"{br_str(BR)}" for BR in i_df["BR"]])
i_df = i_df.assign(MS = [f"{mscale_str(M2, M1, p)}" for (M2, M1, p) in zip(i_df["M2"], i_df["M1"], i_df["pow"])])
i_df = i_df.assign(Ref = [f"{refstr(r)}" for r in i_df["Reference"]])
i_df = i_df.assign(F = [f"" for i in i_df["Reference"]]) # No D reactions have troe parameter.

# Make the final df
i_df.rename(columns = {'ratecoeff':'Rate coefficient'}, inplace = True)
idf_final = i_df.drop(["R1", "R2", "P1", "P2", "M2", "M1", "pow",  "kA", "kB", "kC", "Reference"], axis=1)
idf_final = idf_final[["Num", "reactants", "products",  "BR", "MS", "Rate coefficient", "F", "Ref"]]

# We save the dataframe into a latex table
idf_final.style.format(escape="latex")  
idf_final.to_latex("ReactionTables/ion_reactions_noD.tex", index=False, label="ion_rxns", longtable=True, escape=False)

/tmp/ipykernel_31114/2031514469.py:45: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  idf_final.to_latex("ReactionTables/ion_reactions_noD.tex", index=False, label="ion_rxns", longtable=True, escape=False)


In [40]:
idf_final

,Num,reactants,products,BR,MS,Rate coefficient,F,Ref
0,90,$\mathrm{ArH^+ + C}$,$\mathrm{CH^+ + Ar}$,,,$1.02 \times 10^{-9}$,,Est.
1,91,$\mathrm{ArH^+ + CO}$,$\mathrm{HCO^+ + Ar}$,,,$1.25 \times 10^{-9}$,,A93
2,92,$\mathrm{ArH^+ + CO_2}$,$\mathrm{HCO_2^+ + Ar}$,,,$1.10 \times 10^{-9}$,,A93
3,93,$\mathrm{ArH^+ + e^-}$,$\mathrm{Ar + H}$,,,$1.00 \times 10^{-9}$,,Mi05
4,94,$\mathrm{ArH^+ + H_2}$,$\mathrm{H_3^+ + Ar}$,,,$6.30 \times 10^{-10}$,,A93
...,...,...,...,...,...,...,...,...
267,295,$\mathrm{O^+ + N_2}$,$\mathrm{NO^+ + N}$,,,$4.58 \times 10^{-9}\left(\frac{T_i}{300}\right)^{-1.37}e^{-28.592/T_i}$,,L03
268,296,$\mathrm{O^+ + NO}$,$\mathrm{NO^+ + O}$,,,$8.00 \times 10^{-13}$,,A93
269,297,$\mathrm{O^+ + O_2}$,$\mathrm{O_2^+ + O}$,,,$2.10 \times 10^{-11}$,,A93
270,298a,$\mathrm{O^+ + OH}$,$\mathrm{O_2^+ + H}$,,,$6.24 \times 10^{-9}\left(\frac{T_i}{300}\right)^{-0.5}$,,UMIST
